In [14]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv')

# Função para capturar as previsões e os valores reais de um período específico
def get_predictions_for_period(data, start_date, end_date):
    # Inicializar o modelo
    rf = RandomForestRegressor(n_estimators=50, random_state=42)

    # Converter a coluna 'Date' para datetime se ainda não for
    if data['Date'].dtype != '<M8[ns]':  # <M8[ns] é a representação de datetime64 do numpy
        data['Date'] = pd.to_datetime(data['Date'])

    # Filtrar os dados para o período de treino (antes da data de início)
    train = data[data['Date'] < start_date]
    X_train = train.drop(['Preco_Real', 'Date'], axis=1)
    y_train = train['Preco_Real']

    # Filtrar os dados para o período específico de previsão
    test = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
    X_test = test.drop(['Preco_Real', 'Date'], axis=1)
    y_test = test['Preco_Real']

    # Treinar o modelo
    rf.fit(X_train, y_train)

    # Obter previsões para o período específico
    predictions = rf.predict(X_test)

     # Calcular métricas de desempenho
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    # Retornar os valores reais e previstos junto com as datas e métricas de desempenho
    return test['Date'], y_test, predictions, mse, rmse, mae, r2

# Especificar o período que queremos prever
start_pred_period = pd.Timestamp('2021-07-01')
end_pred_period = pd.Timestamp('2023-07-07')

# Obter previsões e métricas para o período especificado
dates, actual_values, predicted_values, mse, rmse, mae, r2 = get_predictions_for_period(
    df, start_pred_period, end_pred_period
)

# Imprimir os valores reais e previstos para cada data no período especificado
for date, actual, predicted in zip(dates, actual_values, predicted_values):
    print(f"Date: {date.date()}, Actual: {actual}, Predicted: {predicted}")

# Imprimir as métricas de desempenho
print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R^2: {r2}")

Date: 2021-07-01, Actual: 846.16, Predicted: 839.7651999999999
Date: 2021-07-02, Actual: 840.25, Predicted: 838.3510000000002
Date: 2021-07-05, Actual: 838.99, Predicted: 838.2686000000001
Date: 2021-07-06, Actual: 817.13, Predicted: 830.0314000000002
Date: 2021-07-07, Actual: 821.63, Predicted: 829.2542000000003
Date: 2021-07-08, Actual: 837.72, Predicted: 837.4191999999999
Date: 2021-07-09, Actual: 840.43, Predicted: 839.2606
Date: 2021-07-12, Actual: 854.69, Predicted: 839.5270000000003
Date: 2021-07-13, Actual: 836.9, Predicted: 836.8150000000003
Date: 2021-07-14, Actual: 850.48, Predicted: 844.7268000000003
Date: 2021-07-15, Actual: 843.71, Predicted: 841.7512000000002
Date: 2021-07-16, Actual: 875.45, Predicted: 854.2517999999998
Date: 2021-07-19, Actual: 866.41, Predicted: 839.7661999999998
Date: 2021-07-20, Actual: 902.64, Predicted: 847.9271999999997
Date: 2021-07-21, Actual: 960.5, Predicted: 848.6819999999996
Date: 2021-07-22, Actual: 1028.35, Predicted: 847.0529999999999
Da

7 DIAS

In [18]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Função para realizar walk-forward testing
def walk_forward_testing(data, train_start, test_period):
    # Inicializar o modelo
    rf = RandomForestRegressor(n_estimators=10, random_state=42)
    
    predictions = []
    actuals = []
    dates = []

    # Walk-forward over each step in the test period
    for start_date in pd.date_range(train_start, periods=test_period, freq='MS'):
        end_date = start_date + pd.offsets.MonthEnd(0)
        
        # Dados de treino até a data de início
        train = data[data['Date'] < start_date]
        X_train = train.drop(['Preco_Real', 'Date'], axis=1)
        y_train = train['Preco_Real']
        
        # Dados de teste para o mês atual
        test = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
        X_test = test.drop(['Preco_Real', 'Date'], axis=1)
        y_test = test['Preco_Real']
        
        # Treinar o modelo e fazer previsões
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        # Armazenar previsões, valores reais e datas
        predictions.extend(y_pred)
        actuals.extend(y_test)
        dates.extend(test['Date'])
    
    return dates, actuals, predictions

# Definir o período de treino inicial e o número de passos a avançar
train_start_date = pd.Timestamp('2022-01-01')
test_period = 19  # Número de meses para avançar

# Realizar o walk-forward testing
dates, actuals, predictions = walk_forward_testing(df, train_start_date, test_period)

# Calcular as métricas de desempenho
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

# Preparar o DataFrame com as previsões e os valores reais
predictions_df = pd.DataFrame({
    'Date': dates,
    'Actual': actuals,
    'Predicted': predictions
})

# Imprimir as métricas de desempenho
print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R^2: {r2}")

# Filtrar o DataFrame para o período desejado para a previsão
desired_prediction_period = predictions_df[
    (predictions_df['Date'] >= '2023-07-01') &
    (predictions_df['Date'] <= '2023-07-09')
]

# Exibir as previsões para o período desejado
print(desired_prediction_period)


MSE: 4263.674826194937, RMSE: 65.29682095014226, MAE: 43.89822025316456, R^2: 0.8718219585628237
          Date  Actual  Predicted
374 2023-07-03  824.48    842.718
375 2023-07-04  824.70    832.187
376 2023-07-05  822.25    839.895
377 2023-07-06  829.41    838.838
378 2023-07-07  829.19    834.814


In [19]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/clima.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Função para realizar walk-forward testing
def walk_forward_testing(data, train_start, test_period):
    # Inicializar o modelo
    rf = RandomForestRegressor(n_estimators=10, random_state=42)
    
    predictions = []
    actuals = []
    dates = []

    # Walk-forward over each step in the test period
    for start_date in pd.date_range(train_start, periods=test_period, freq='MS'):
        end_date = start_date + pd.offsets.MonthEnd(0)
        
        # Dados de treino até a data de início
        train = data[data['Date'] < start_date]
        X_train = train.drop(['Preco_Real', 'Date'], axis=1)
        y_train = train['Preco_Real']
        
        # Dados de teste para o mês atual
        test = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
        X_test = test.drop(['Preco_Real', 'Date'], axis=1)
        y_test = test['Preco_Real']
        
        # Treinar o modelo e fazer previsões
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        # Armazenar previsões, valores reais e datas
        predictions.extend(y_pred)
        actuals.extend(y_test)
        dates.extend(test['Date'])
    
    return dates, actuals, predictions

# Definir o período de treino inicial e o número de passos a avançar
train_start_date = pd.Timestamp('2022-01-01')
test_period = 19  # Número de meses para avançar

# Realizar o walk-forward testing
dates, actuals, predictions = walk_forward_testing(df, train_start_date, test_period)

# Calcular as métricas de desempenho
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

# Preparar o DataFrame com as previsões e os valores reais
predictions_df = pd.DataFrame({
    'Date': dates,
    'Actual': actuals,
    'Predicted': predictions
})

# Imprimir as métricas de desempenho
print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R^2: {r2}")

# Filtrar o DataFrame para o período desejado para a previsão
desired_prediction_period = predictions_df[
    (predictions_df['Date'] >= '2023-07-01') &
    (predictions_df['Date'] <= '2023-07-09')
]

# Exibir as previsões para o período desejado
print(desired_prediction_period)


MSE: 300927.8826980152, RMSE: 548.5689406975346, MAE: 480.87197974683545, R^2: -8.046737424976497
          Date  Actual  Predicted
374 2023-07-03  824.48    849.248
375 2023-07-04  824.70    854.996
376 2023-07-05  822.25    854.011
377 2023-07-06  829.41    862.147
378 2023-07-07  829.19   1160.413


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Função para realizar walk-forward testing
def walk_forward_testing(data, train_start, test_period):
    # Inicializar o modelo
    rf = RandomForestRegressor(n_estimators=10, random_state=42)
    
    predictions = []
    actuals = []
    dates = []

    # Walk-forward over each step in the test period
    for start_date in pd.date_range(train_start, periods=test_period, freq='MS'):
        end_date = start_date + pd.offsets.MonthEnd(0)
        
        # Dados de treino até a data de início
        train = data[data['Date'] < start_date]
        X_train = train.drop(['Preco_Real', 'Date'], axis=1)
        y_train = train['Preco_Real']
        
        # Dados de teste para o mês atual
        test = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
        X_test = test.drop(['Preco_Real', 'Date'], axis=1)
        y_test = test['Preco_Real']
        
        # Treinar o modelo e fazer previsões
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        # Armazenar previsões, valores reais e datas
        predictions.extend(y_pred)
        actuals.extend(y_test)
        dates.extend(test['Date'])
    
    return dates, actuals, predictions

# Definir o período de treino inicial e o número de passos a avançar
train_start_date = pd.Timestamp('2022-01-01')
test_period = 19  # Número de meses para avançar

# Realizar o walk-forward testing
dates, actuals, predictions = walk_forward_testing(df, train_start_date, test_period)

# Calcular as métricas de desempenho
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

# Preparar o DataFrame com as previsões e os valores reais
predictions_df = pd.DataFrame({
    'Date': dates,
    'Actual': actuals,
    'Predicted': predictions
})

# Imprimir as métricas de desempenho
print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R^2: {r2}")

# Filtrar o DataFrame para o período desejado para a previsão
desired_prediction_period = predictions_df[
    (predictions_df['Date'] >= '2023-07-01') &
    (predictions_df['Date'] <= '2023-07-09')
]

# Exibir as previsões para o período desejado
print(desired_prediction_period)


In [20]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibovespa.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Função para realizar walk-forward testing
def walk_forward_testing(data, train_start, test_period):
    # Inicializar o modelo
    rf = RandomForestRegressor(n_estimators=10, random_state=42)
    
    predictions = []
    actuals = []
    dates = []

    # Walk-forward over each step in the test period
    for start_date in pd.date_range(train_start, periods=test_period, freq='MS'):
        end_date = start_date + pd.offsets.MonthEnd(0)
        
        # Dados de treino até a data de início
        train = data[data['Date'] < start_date]
        X_train = train.drop(['Preco_Real', 'Date'], axis=1)
        y_train = train['Preco_Real']
        
        # Dados de teste para o mês atual
        test = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
        X_test = test.drop(['Preco_Real', 'Date'], axis=1)
        y_test = test['Preco_Real']
        
        # Treinar o modelo e fazer previsões
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        # Armazenar previsões, valores reais e datas
        predictions.extend(y_pred)
        actuals.extend(y_test)
        dates.extend(test['Date'])
    
    return dates, actuals, predictions

# Definir o período de treino inicial e o número de passos a avançar
train_start_date = pd.Timestamp('2022-01-01')
test_period = 19  # Número de meses para avançar

# Realizar o walk-forward testing
dates, actuals, predictions = walk_forward_testing(df, train_start_date, test_period)

# Calcular as métricas de desempenho
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

# Preparar o DataFrame com as previsões e os valores reais
predictions_df = pd.DataFrame({
    'Date': dates,
    'Actual': actuals,
    'Predicted': predictions
})

# Imprimir as métricas de desempenho
print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R^2: {r2}")

# Filtrar o DataFrame para o período desejado para a previsão
desired_prediction_period = predictions_df[
    (predictions_df['Date'] >= '2023-07-01') &
    (predictions_df['Date'] <= '2023-07-09')
]

# Exibir as previsões para o período desejado
print(desired_prediction_period)


MSE: 116081.29382544672, RMSE: 340.7070498616762, MAE: 253.96814451476789, R^2: -2.489729751111855
          Date  Actual  Predicted
374 2023-07-03  824.48    781.674
375 2023-07-04  824.70    615.539
376 2023-07-05  822.25    985.187
377 2023-07-06  829.41   1042.327
378 2023-07-07  829.19    958.433


In [21]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/selic.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Função para realizar walk-forward testing
def walk_forward_testing(data, train_start, test_period):
    # Inicializar o modelo
    rf = RandomForestRegressor(n_estimators=10, random_state=42)
    
    predictions = []
    actuals = []
    dates = []

    # Walk-forward over each step in the test period
    for start_date in pd.date_range(train_start, periods=test_period, freq='MS'):
        end_date = start_date + pd.offsets.MonthEnd(0)
        
        # Dados de treino até a data de início
        train = data[data['Date'] < start_date]
        X_train = train.drop(['Preco_Real', 'Date'], axis=1)
        y_train = train['Preco_Real']
        
        # Dados de teste para o mês atual
        test = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
        X_test = test.drop(['Preco_Real', 'Date'], axis=1)
        y_test = test['Preco_Real']
        
        # Treinar o modelo e fazer previsões
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        # Armazenar previsões, valores reais e datas
        predictions.extend(y_pred)
        actuals.extend(y_test)
        dates.extend(test['Date'])
    
    return dates, actuals, predictions

# Definir o período de treino inicial e o número de passos a avançar
train_start_date = pd.Timestamp('2022-01-01')
test_period = 19  # Número de meses para avançar

# Realizar o walk-forward testing
dates, actuals, predictions = walk_forward_testing(df, train_start_date, test_period)

# Calcular as métricas de desempenho
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

# Preparar o DataFrame com as previsões e os valores reais
predictions_df = pd.DataFrame({
    'Date': dates,
    'Actual': actuals,
    'Predicted': predictions
})

# Imprimir as métricas de desempenho
print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R^2: {r2}")

# Filtrar o DataFrame para o período desejado para a previsão
desired_prediction_period = predictions_df[
    (predictions_df['Date'] >= '2023-07-01') &
    (predictions_df['Date'] <= '2023-07-09')
]

# Exibir as previsões para o período desejado
print(desired_prediction_period)


MSE: 19835.384471812664, RMSE: 140.83814991618095, MAE: 102.90004050632912, R^2: 0.4036926275122832
          Date  Actual  Predicted
374 2023-07-03  824.48   1050.601
375 2023-07-04  824.70   1046.083
376 2023-07-05  822.25   1049.163
377 2023-07-06  829.41   1118.576
378 2023-07-07  829.19   1039.013


1 DIA

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/selic.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Função para realizar walk-forward testing
def walk_forward_testing(data, train_start, test_period):
    # Inicializar o modelo
    rf = RandomForestRegressor(n_estimators=10, random_state=42)
    
    predictions = []
    actuals = []
    dates = []

    # Walk-forward over each step in the test period
    for start_date in pd.date_range(train_start, periods=test_period, freq='MS'):
        end_date = start_date + pd.offsets.MonthEnd(0)
        
        # Dados de treino até a data de início
        train = data[data['Date'] < start_date]
        X_train = train.drop(['Preco_Real', 'Date'], axis=1)
        y_train = train['Preco_Real']
        
        # Dados de teste para o mês atual
        test = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
        X_test = test.drop(['Preco_Real', 'Date'], axis=1)
        y_test = test['Preco_Real']
        
        # Treinar o modelo e fazer previsões
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        # Armazenar previsões, valores reais e datas
        predictions.extend(y_pred)
        actuals.extend(y_test)
        dates.extend(test['Date'])
    
    return dates, actuals, predictions

# Definir o período de treino inicial e o número de passos a avançar
train_start_date = pd.Timestamp('2022-01-01')
test_period = 19  # Número de meses para avançar

# Realizar o walk-forward testing
dates, actuals, predictions = walk_forward_testing(df, train_start_date, test_period)

# Calcular as métricas de desempenho
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

# Preparar o DataFrame com as previsões e os valores reais
predictions_df = pd.DataFrame({
    'Date': dates,
    'Actual': actuals,
    'Predicted': predictions
})

# Imprimir as métricas de desempenho
print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R^2: {r2}")

# Filtrar o DataFrame para o período desejado para a previsão
desired_prediction_period = predictions_df[
    (predictions_df['Date'] == '2023-07-06') 
]

# Exibir as previsões para o período desejado
print(desired_prediction_period)


MSE: 19835.384471812664, RMSE: 140.83814991618095, MAE: 102.90004050632912, R^2: 0.4036926275122832
          Date  Actual  Predicted
377 2023-07-06  829.41   1118.576


In [29]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibovespa.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Função para realizar walk-forward testing
def walk_forward_testing(data, train_start, test_period):
    # Inicializar o modelo
    rf = RandomForestRegressor(n_estimators=10, random_state=42)
    
    predictions = []
    actuals = []
    dates = []

    # Walk-forward over each step in the test period
    for start_date in pd.date_range(train_start, periods=test_period, freq='MS'):
        end_date = start_date + pd.offsets.MonthEnd(0)
        
        # Dados de treino até a data de início
        train = data[data['Date'] < start_date]
        X_train = train.drop(['Preco_Real', 'Date'], axis=1)
        y_train = train['Preco_Real']
        
        # Dados de teste para o mês atual
        test = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
        X_test = test.drop(['Preco_Real', 'Date'], axis=1)
        y_test = test['Preco_Real']
        
        # Treinar o modelo e fazer previsões
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        # Armazenar previsões, valores reais e datas
        predictions.extend(y_pred)
        actuals.extend(y_test)
        dates.extend(test['Date'])
    
    return dates, actuals, predictions

# Definir o período de treino inicial e o número de passos a avançar
train_start_date = pd.Timestamp('2022-01-01')
test_period = 19  # Número de meses para avançar

# Realizar o walk-forward testing
dates, actuals, predictions = walk_forward_testing(df, train_start_date, test_period)

# Calcular as métricas de desempenho
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

# Preparar o DataFrame com as previsões e os valores reais
predictions_df = pd.DataFrame({
    'Date': dates,
    'Actual': actuals,
    'Predicted': predictions
})

# Imprimir as métricas de desempenho
print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R^2: {r2}")

# Filtrar o DataFrame para o período desejado para a previsão
desired_prediction_period = predictions_df[
    (predictions_df['Date'] == '2023-07-06') 
]

# Exibir as previsões para o período desejado
print(desired_prediction_period)


MSE: 116081.29382544672, RMSE: 340.7070498616762, MAE: 253.96814451476789, R^2: -2.489729751111855
          Date  Actual  Predicted
377 2023-07-06  829.41   1042.327


In [30]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/clima.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Função para realizar walk-forward testing
def walk_forward_testing(data, train_start, test_period):
    # Inicializar o modelo
    rf = RandomForestRegressor(n_estimators=10, random_state=42)
    
    predictions = []
    actuals = []
    dates = []

    # Walk-forward over each step in the test period
    for start_date in pd.date_range(train_start, periods=test_period, freq='MS'):
        end_date = start_date + pd.offsets.MonthEnd(0)
        
        # Dados de treino até a data de início
        train = data[data['Date'] < start_date]
        X_train = train.drop(['Preco_Real', 'Date'], axis=1)
        y_train = train['Preco_Real']
        
        # Dados de teste para o mês atual
        test = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
        X_test = test.drop(['Preco_Real', 'Date'], axis=1)
        y_test = test['Preco_Real']
        
        # Treinar o modelo e fazer previsões
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        # Armazenar previsões, valores reais e datas
        predictions.extend(y_pred)
        actuals.extend(y_test)
        dates.extend(test['Date'])
    
    return dates, actuals, predictions

# Definir o período de treino inicial e o número de passos a avançar
train_start_date = pd.Timestamp('2022-01-01')
test_period = 19  # Número de meses para avançar

# Realizar o walk-forward testing
dates, actuals, predictions = walk_forward_testing(df, train_start_date, test_period)

# Calcular as métricas de desempenho
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

# Preparar o DataFrame com as previsões e os valores reais
predictions_df = pd.DataFrame({
    'Date': dates,
    'Actual': actuals,
    'Predicted': predictions
})

# Imprimir as métricas de desempenho
print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R^2: {r2}")

# Filtrar o DataFrame para o período desejado para a previsão
desired_prediction_period = predictions_df[
    (predictions_df['Date'] == '2023-07-06') 
]

# Exibir as previsões para o período desejado
print(desired_prediction_period)


MSE: 300927.8826980152, RMSE: 548.5689406975346, MAE: 480.87197974683545, R^2: -8.046737424976497
          Date  Actual  Predicted
377 2023-07-06  829.41    862.147


In [31]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Função para realizar walk-forward testing
def walk_forward_testing(data, train_start, test_period):
    # Inicializar o modelo
    rf = RandomForestRegressor(n_estimators=10, random_state=42)
    
    predictions = []
    actuals = []
    dates = []

    # Walk-forward over each step in the test period
    for start_date in pd.date_range(train_start, periods=test_period, freq='MS'):
        end_date = start_date + pd.offsets.MonthEnd(0)
        
        # Dados de treino até a data de início
        train = data[data['Date'] < start_date]
        X_train = train.drop(['Preco_Real', 'Date'], axis=1)
        y_train = train['Preco_Real']
        
        # Dados de teste para o mês atual
        test = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
        X_test = test.drop(['Preco_Real', 'Date'], axis=1)
        y_test = test['Preco_Real']
        
        # Treinar o modelo e fazer previsões
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        # Armazenar previsões, valores reais e datas
        predictions.extend(y_pred)
        actuals.extend(y_test)
        dates.extend(test['Date'])
    
    return dates, actuals, predictions

# Definir o período de treino inicial e o número de passos a avançar
train_start_date = pd.Timestamp('2022-01-01')
test_period = 19  # Número de meses para avançar

# Realizar o walk-forward testing
dates, actuals, predictions = walk_forward_testing(df, train_start_date, test_period)

# Calcular as métricas de desempenho
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

# Preparar o DataFrame com as previsões e os valores reais
predictions_df = pd.DataFrame({
    'Date': dates,
    'Actual': actuals,
    'Predicted': predictions
})

# Imprimir as métricas de desempenho
print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R^2: {r2}")

# Filtrar o DataFrame para o período desejado para a previsão
desired_prediction_period = predictions_df[
    (predictions_df['Date'] == '2023-07-06') 
]

# Exibir as previsões para o período desejado
print(desired_prediction_period)


MSE: 21772.005362298736, RMSE: 147.55339834208746, MAE: 58.05672405063293, R^2: 0.34547236380367463
          Date  Actual  Predicted
377 2023-07-06  829.41    835.176


In [32]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregar os dados
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv')
df['Date'] = pd.to_datetime(df['Date'])

# Função para realizar walk-forward testing
def walk_forward_testing(data, train_start, test_period):
    # Inicializar o modelo
    rf = RandomForestRegressor(n_estimators=10, random_state=42)
    
    predictions = []
    actuals = []
    dates = []

    # Walk-forward over each step in the test period
    for start_date in pd.date_range(train_start, periods=test_period, freq='MS'):
        end_date = start_date + pd.offsets.MonthEnd(0)
        
        # Dados de treino até a data de início
        train = data[data['Date'] < start_date]
        X_train = train.drop(['Preco_Real', 'Date'], axis=1)
        y_train = train['Preco_Real']
        
        # Dados de teste para o mês atual
        test = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
        X_test = test.drop(['Preco_Real', 'Date'], axis=1)
        y_test = test['Preco_Real']
        
        # Treinar o modelo e fazer previsões
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        # Armazenar previsões, valores reais e datas
        predictions.extend(y_pred)
        actuals.extend(y_test)
        dates.extend(test['Date'])
    
    return dates, actuals, predictions

# Definir o período de treino inicial e o número de passos a avançar
train_start_date = pd.Timestamp('2022-01-01')
test_period = 19  # Número de meses para avançar

# Realizar o walk-forward testing
dates, actuals, predictions = walk_forward_testing(df, train_start_date, test_period)

# Calcular as métricas de desempenho
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

# Preparar o DataFrame com as previsões e os valores reais
predictions_df = pd.DataFrame({
    'Date': dates,
    'Actual': actuals,
    'Predicted': predictions
})

# Imprimir as métricas de desempenho
print(f"MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R^2: {r2}")

# Filtrar o DataFrame para o período desejado para a previsão
desired_prediction_period = predictions_df[
    (predictions_df['Date'] == '2023-07-06') 
]

# Exibir as previsões para o período desejado
print(desired_prediction_period)


MSE: 4263.674826194937, RMSE: 65.29682095014226, MAE: 43.89822025316456, R^2: 0.8718219585628237
          Date  Actual  Predicted
377 2023-07-06  829.41    838.838
